<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/project2_individual/financial_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to classify records as one category or an other of financial

In [4]:
!ls

/content


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# https://www.kaggle.com/code/ahmadalijamali/twitter-financial-news-nlp-analysis-and-prediction/notebook
import pandas as pd

#df=pd.read_csv("./data/train_data.csv")
df=pd.read_csv("/content/drive/MyDrive/ColabNotebooks/ml_200_individual/train_data.csv")
df.head()
#df.sample(10)
df["label"].unique()


array([ 0,  2,  9,  8,  4,  5,  6,  1,  7, 10, 11, 12, 13, 14, 15, 17, 16,
       18, 19,  3])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

countGroupBy=df.groupby("label").count()

counts = df['label'].value_counts()
# countGroupBy.reset_index()
counts.columns = ['label', 'count']
#pd.options.plotting.backend = 'matplotlib'
# countGroupBy.plot.bar(x="label",y="text")


In [ ]:
countGroupBy.plot(x="label",y="text")

KeyError: 'label'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df.plot(x="label",y="coun")


ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.